In [51]:
filepath = './datasets'
gauge_path = filepath+'/gauge.csv'
radar_path = filepath+'/radar'

In [52]:
from tqdm import tqdm_notebook as tqdm
from scipy.stats import pearsonr

In [53]:
import pandas as pd
import numpy as np
import os

In [54]:
df_gauge = pd.read_csv(gauge_path)

In [55]:
df_result = pd.DataFrame()
train_week = []
for filename in tqdm(os.listdir(radar_path)):
    week = filename.split('.')[0][-2:]
    train_week.append(str(week))
    full_radar_path = os.path.join(radar_path, filename)
    df_radar = pd.read_csv(full_radar_path)
    radar_obs = df_radar.values[:,2:]
    radar_arange = np.arange(0, 34)
    radar_mean = np.matmul(radar_obs, radar_arange) / 2016.
    if 'latlong' not in df_result.columns:
        df_result['latlong'] = df_radar['lat'] * 1000 + df_radar['long']
    assert np.allclose(df_result['latlong'].values, df_radar['lat'] * 1000 + df_radar['long'])
    df_result['mean_week_'+str(week)] = radar_mean

HBox(children=(IntProgress(value=0, max=31), HTML(value='')))

In [56]:
train_week

['18',
 '36',
 '38',
 '31',
 '44',
 '16',
 '41',
 '21',
 '47',
 '33',
 '48',
 '17',
 '20',
 '51',
 '39',
 '35',
 '22',
 '15',
 '37',
 '29',
 '45',
 '13',
 '26',
 '19',
 '23',
 '49',
 '50',
 '46',
 '32',
 '24',
 '28']

In [57]:
df_result = df_result[sorted(df_result.columns)]

In [58]:
df_result.head()

,latlong,mean_week_13,mean_week_15,mean_week_16,mean_week_17,mean_week_18,mean_week_19,mean_week_20,mean_week_21,mean_week_22,...,mean_week_39,mean_week_41,mean_week_44,mean_week_45,mean_week_46,mean_week_47,mean_week_48,mean_week_49,mean_week_50,mean_week_51
0,0,0.675099,0.382937,0.535714,0.815476,0.248016,0.237599,0.416667,0.908730,0.306052,...,0.193948,0.994544,0.695933,1.006944,1.025298,0.670139,0.461310,0.645337,0.406250,0.157242
1,1,0.714782,0.382440,0.493552,0.841270,0.258433,0.266369,0.450397,0.963790,0.289187,...,0.164683,0.980655,0.738591,1.010417,1.021825,0.635913,0.457341,0.637897,0.339782,0.183532
2,2,0.742063,0.411706,0.519345,0.859127,0.235615,0.260417,0.466270,0.968254,0.324901,...,0.145833,0.958333,0.691468,1.002480,1.012897,0.616071,0.469742,0.611607,0.336806,0.154266
3,3,0.746032,0.380456,0.522321,0.887897,0.247520,0.278274,0.474702,0.954365,0.307540,...,0.157738,0.932044,0.755952,1.075893,1.037202,0.561012,0.494048,0.574405,0.350694,0.157242
4,4,0.745536,0.380456,0.519345,0.860119,0.248016,0.277778,0.472222,0.950893,0.304563,...,0.155258,0.923115,0.771825,1.070933,1.038690,0.553075,0.499008,0.573909,0.352183,0.154266


In [59]:
df_gauge = pd.read_csv(gauge_path)

In [60]:
len(np.where(pd.isnull(df_gauge))[0])

357

In [61]:
df_gauge['Year'] = df_gauge['Year'].astype(str)
df_gauge['Week'] = df_gauge['Week'].astype(str)
df_gauge = df_gauge.apply(lambda x: x.fillna(x[2:].mean()),axis=1)

In [62]:
train_week = sorted(train_week)

In [63]:
train_bool = df_gauge['Week'].apply(lambda x : x in train_week)
train_df_gauge = df_gauge[train_bool]

In [64]:
train_df_gauge_array = train_df_gauge.values[:, 2:]

In [65]:
train_df_gauge.head()

,Year,Week,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
12,2017,13,36.8,30.0,48.2,78.6,67.4,45.6,77.6,21.6,...,37.800000,34.0,43.645455,19.4,25.2,26.6,4.8,57.000000,107.000000,54.000000
14,2017,15,43.8,90.4,82.6,13.2,45.8,86.6,34.6,38.4,...,46.000000,99.0,66.773810,66.6,72.9,76.2,82.4,53.600000,21.800000,66.773810
15,2017,16,98.8,134.0,68.2,74.8,77.7,86.4,37.8,102.6,...,97.579487,139.0,97.579487,58.6,69.7,68.2,162.6,97.579487,43.000000,97.579487
16,2017,17,100.2,56.0,82.8,69.2,127.3,84.6,78.0,24.8,...,64.136842,46.2,89.200000,28.0,38.7,112.6,17.8,34.600000,64.136842,32.400000
17,2017,18,2.0,76.0,1.6,1.2,54.5,25.2,30.4,49.8,...,36.884211,24.8,36.884211,29.2,25.5,109.8,71.6,14.000000,36.884211,9.600000


In [66]:
train_df_gauge_array.shape

(31, 50)

In [67]:
df_result['latlong'].values

array([     0,      1,      2, ..., 479477, 479478, 479479])

In [69]:
print (np.where(pd.isnull(train_df_gauge_array)))
print (np.where(pd.isnull(df_result)))

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [ ]:
df_corr_result = pd.DataFrame()
df_corr_result['latlong'] = df_result['latlong'].values
for station in tqdm(range(train_df_gauge_array.shape[1])):
    station_values = train_df_gauge_array[:, station]
    total_latlong = []
    for idx, latlong in tqdm(df_result.iterrows()):
        latlong_values = latlong.values[1:]
        correlation = np.corrcoef(station_values.tolist(), latlong_values.tolist())[0][1]
        total_latlong.append(correlation)
    df_corr_result['corr_station_'+str(station)] = total_latlong

In [46]:
df_corr_result.to_csv('adit_csv')

In [32]:
df_corr_result = pd.read_csv('adit.csv')
del df_corr_result['Unnamed: 0']

In [44]:
df_corr_result.head()

,latlong,corr_station_0,corr_station_1,corr_station_2,corr_station_3,corr_station_4,corr_station_5,corr_station_6,corr_station_7,corr_station_8,...,corr_station_40,corr_station_41,corr_station_42,corr_station_43,corr_station_44,corr_station_45,corr_station_46,corr_station_47,corr_station_48,corr_station_49
0,0,0.468375,0.273661,0.479921,0.491281,0.337894,0.393949,0.412037,0.345347,0.296188,...,0.419544,0.332368,0.513008,0.414793,0.395643,0.329295,0.324398,0.507117,0.380906,0.310547
1,1,0.433552,0.264759,0.450911,0.473669,0.305402,0.360777,0.387895,0.305570,0.275549,...,0.388176,0.294830,0.491740,0.389865,0.370007,0.314823,0.289476,0.474651,0.365256,0.291112
2,2,0.456355,0.272097,0.479214,0.492035,0.327644,0.371132,0.404127,0.310744,0.284599,...,0.407294,0.309539,0.510741,0.390300,0.368940,0.333208,0.298604,0.493650,0.379710,0.302118
3,3,0.465419,0.252170,0.487428,0.505848,0.343355,0.373347,0.420701,0.322977,0.269334,...,0.432371,0.317623,0.515310,0.409838,0.391708,0.329463,0.281805,0.493648,0.402857,0.313750
4,4,0.458610,0.252325,0.481502,0.502371,0.335389,0.372344,0.416009,0.326638,0.271437,...,0.431122,0.318664,0.511574,0.411709,0.395662,0.322257,0.282738,0.494274,0.404142,0.320299


In [45]:
df_corr_result.sort_values(['corr_station_0'],ascending = False).head()

,latlong,corr_station_0,corr_station_1,corr_station_2,corr_station_3,corr_station_4,corr_station_5,corr_station_6,corr_station_7,corr_station_8,...,corr_station_40,corr_station_41,corr_station_42,corr_station_43,corr_station_44,corr_station_45,corr_station_46,corr_station_47,corr_station_48,corr_station_49
115904,241224,0.897161,0.310128,0.813285,0.743815,0.839400,0.702434,0.745927,0.671586,0.312172,...,0.816810,0.714358,0.703024,0.708546,0.664454,0.493947,0.390061,0.836444,0.704287,0.457008
115905,241225,0.889518,0.291900,0.801164,0.727606,0.848222,0.714748,0.732758,0.657291,0.279756,...,0.825018,0.714048,0.690225,0.720705,0.667988,0.513642,0.372627,0.841287,0.690155,0.412839
113987,237227,0.872859,0.199349,0.766339,0.781460,0.782592,0.612099,0.786879,0.665462,0.207083,...,0.731076,0.653738,0.642224,0.705022,0.649218,0.484623,0.260560,0.748928,0.636124,0.280807
116862,243222,0.872584,0.281486,0.791896,0.719154,0.803683,0.692231,0.717802,0.643570,0.288233,...,0.815868,0.704613,0.723659,0.730904,0.673126,0.469931,0.421651,0.839041,0.637604,0.432010
113504,236224,0.867482,0.264679,0.748601,0.765155,0.786952,0.618998,0.759675,0.680713,0.250499,...,0.738939,0.676045,0.650842,0.746825,0.680182,0.454156,0.328116,0.773486,0.632247,0.279572


In [37]:
len(np.where(pd.isnull(df_corr_result))[0])

24600

In [38]:
latlong = df_corr_result.values[:,0]

In [39]:
df_corr_result_array = df_corr_result.values[:,1:]
df_corr_result_array.shape

(230400, 50)

In [40]:
def to_lat(latlong):
    return int(latlong // 1000)

def to_long(latlong):
    return int(latlong % 1000)

In [75]:
dict_result = {}
for station in range(df_corr_result_array.shape[1]):
    values = df_corr_result_array[:, station]
    index = np.nanargmax(values)
    dict_result[str(station)] = {
    'lat' : to_lat(latlong[index]),
    'long': to_long(latlong[index]),
    'latlong': latlong[index]
    }

In [76]:
import json
with open('station_location.json', 'w') as outfile:
    json.dump(dict_result, outfile)

In [77]:
dict_result

{'0': {'lat': 241, 'latlong': 241224.0, 'long': 224},
 '1': {'lat': 241, 'latlong': 241102.0, 'long': 102},
 '10': {'lat': 256, 'latlong': 256144.0, 'long': 144},
 '11': {'lat': 242, 'latlong': 242154.0, 'long': 154},
 '12': {'lat': 188, 'latlong': 188156.0, 'long': 156},
 '13': {'lat': 253, 'latlong': 253201.0, 'long': 201},
 '14': {'lat': 245, 'latlong': 245170.0, 'long': 170},
 '15': {'lat': 251, 'latlong': 251157.0, 'long': 157},
 '16': {'lat': 253, 'latlong': 253201.0, 'long': 201},
 '17': {'lat': 232, 'latlong': 232248.0, 'long': 248},
 '18': {'lat': 273, 'latlong': 273181.0, 'long': 181},
 '19': {'lat': 230, 'latlong': 230179.0, 'long': 179},
 '2': {'lat': 253, 'latlong': 253221.0, 'long': 221},
 '20': {'lat': 214, 'latlong': 214116.0, 'long': 116},
 '21': {'lat': 230, 'latlong': 230179.0, 'long': 179},
 '22': {'lat': 202, 'latlong': 202117.0, 'long': 117},
 '23': {'lat': 236, 'latlong': 236247.0, 'long': 247},
 '24': {'lat': 229, 'latlong': 229160.0, 'long': 160},
 '25': {'lat'